In [18]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
distance_calculation_df = (
    spark
    .read
    .format("mongodb")
    .option("database", "enhancer3d")
    .option("collection", "distance_calculation")
    .load()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
install_deps(['numpy', 'matplotlib', 'pandas'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
from pyspark.sql import functions as F, types as T
import numpy as np

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))

results_GM12878_Deni_chr1_74280_3521135_df = (
    distance_calculation_df
    .select('_id.*', '*')
    .drop('_id')
    .where("project_id == 'GM12878_demo_project' and ensemble_id == 'results_GM12878_Deni_chr1_74280_3521135'")
    .alias("ref")
)

results_GM12878_Deni_mod_chr1_74280_3521135_df = (
    distance_calculation_df
    .select('_id.*', '*')
    .drop('_id')
    .where("project_id == 'GM12878_demo_project' and ensemble_id == 'results_GM12878_Deni_mod_chr1_74280_3521135'")
    .alias("mod")
)

results_GM12878_Deni_chr1_74280_3521135_joined_with_mod_df = (
    results_GM12878_Deni_chr1_74280_3521135_df
    .join(
        results_GM12878_Deni_mod_chr1_74280_3521135_df,
        on=['gene_id', 'enh_id'],
        how='inner'
    )
    .select(
        'gene_id', 
        'enh_id',

        F.col('ref.enh_score').alias('enh_score'),
        
        F.col('ref.dist').alias('ref_dist'),
        F.col('ref.avg_dist').alias('ref_dist_avg'),
        var(F.col('ref.dist')).alias('ref_dist_var'),

        F.col('mod.dist').alias('mod_dist'),
        F.col('mod.avg_dist').alias('mod_dist_avg'),
        var(F.col('mod.dist')).alias('mod_dist_var'),
        
        diff(F.col('ref.dist'), F.col('mod.dist')).alias('dist_diff'),
        avg(F.col('dist_diff')).alias('dist_diff_avg'),
        var(F.col('dist_diff')).alias('dist_diff_var')
    )
)

results_GM12878_Deni_chr1_74280_3521135_joined_with_mod_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13099

In [87]:
results_pandas = results_GM12878_Deni_chr1_74280_3521135_joined_with_mod_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
results_pandas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  gene_id                enh_id  ...  dist_diff_avg dist_diff_var
0      ENSG00000008128.23  chr1:1001810-1003120  ...       2.929620      3.635421
1      ENSG00000008128.23  chr1:1134330-1137600  ...       5.708063     14.787756
2      ENSG00000008128.23  chr1:1838510-1839300  ...       6.756561     27.504060
3      ENSG00000008128.23  chr1:1839320-1841260  ...       6.820991     27.754883
4      ENSG00000008128.23  chr1:2573380-2575610  ...       3.347530      6.081045
...                   ...                   ...  ...            ...           ...
13094   ENSG00000289610.1  chr1:1353070-1353790  ...       2.031172      2.709765
13095   ENSG00000289610.1  chr1:1795730-1796590  ...       1.202879      0.832562
13096   ENSG00000289610.1  chr1:1839320-1841260  ...       2.800798      4.323006
13097   ENSG00000289610.1  chr1:2064070-2065430  ...       1.791814      1.490359
13098   ENSG00000289610.1  chr1:2245160-2249020  ...       3.796132      7.343721

[13099 rows x 1